In [ ]:
import sys, numpy, pandas as pd, sklearn, scipy, matplotlib, seaborn, mlflow
print("Python:", sys.version.split()[0])
print("numpy:", numpy.__version__)
print("pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("seaborn:", seaborn.__version__)
print("mlflow:", mlflow.__version__)

In [ ]:
import os, pathlib, sys
print("CWD:", os.getcwd())
print("Contenido:", os.listdir())

In [ ]:
print("✔️ Run registrada en MLflow")
import mlflow
from sklearn import metrics
import pathlib

# Conectar con tu MLflow local
mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("Equipo24-MER")

with mlflow.start_run(run_name="baseline-notebook"):
    mlflow.log_param("model", "LogisticRegression")   # 👉 cámbialo si usaste otro
    mlflow.log_param("features", "all_columns")

    # 👉 Ajusta y_test / y_pred según tu notebook
    acc = metrics.accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", acc)

    # Guarda una muestra como artifact
    pathlib.Path("artifacts").mkdir(exist_ok=True)
    df.head(50).to_csv("artifacts/sample_preview.csv", index=False)
    mlflow.log_artifact("artifacts/sample_preview.csv")

print("✔️ Run registrada en MLflow")

In [ ]:
# ⚠️ Asegúrate de que 'df' está en memoria y que la columna objetivo se llama "Class".
# Si tu target tiene otro nombre, cámbialo donde dice target_col = "Class"

import mlflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pandas as pd
import numpy as np
import pathlib

import pandas as pd
df = pd.read_csv("Acoustic Features.csv")
df.head()

target_col = "Class"   # <-- cámbialo si tu target tiene otro nombre

# --- Prep rápida ---
y = df[target_col].copy()
X = df.drop(columns=[target_col]).copy()

# Encode etiquetas si son strings
if y.dtype == "object":
    le = LabelEncoder()
    y = le.fit_transform(y)

# Relleno simple por si hay NaN
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

# --- Modelo baseline ---
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_sc, y_train)
y_pred = clf.predict(X_test_sc)

acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

# --- Log en MLflow ---
mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("Equipo24-MER")

with mlflow.start_run(run_name="baseline-local"):
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.log_metric("accuracy", float(acc))

    # guarda una muestra como artefacto
    pathlib.Path("artifacts").mkdir(exist_ok=True)
    df.head(50).to_csv("artifacts/sample_preview.csv", index=False)
    mlflow.log_artifact("artifacts/sample_preview.csv")

print("✔️ Run registrada en MLflow")